# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT',
 'HDFCBANK',
 'HDFCSML250',
 'INDIGOPNTS',
 'ITBEES',
 'JUBLFOOD',
 'NH']

In [4]:
STOCK_SYMBOL = stock_symbols[1]
STOCK_SYMBOL

'HDFCBANK'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df.columns = [
    'Date', 'Series',
    'Open', 'High', 'Low', 'Prev Close', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
] + (["PE"] if "PE" in stock_df.columns else [])
stock_df

,Date,Series,Open,High,Low,Prev Close,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE
0,2020-01-01,EQ,1276.10,1280.00,1270.60,1272.10,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185
1,2020-01-02,EQ,1279.00,1288.00,1279.00,1278.60,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384
2,2020-01-03,EQ,1282.20,1285.00,1263.60,1286.75,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937
3,2020-01-06,EQ,1260.00,1261.80,1236.00,1268.40,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267
4,2020-01-07,EQ,1258.90,1271.45,1252.25,1240.95,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,2024-07-01,EQ,1680.00,1707.30,1680.00,1683.80,1704.20,1705.20,1699.74,1757.5,1363.55,10593512,1.800619e+10,355235,20.221
1118,2024-07-02,EQ,1715.00,1734.90,1702.75,1705.20,1728.00,1730.60,1722.46,1757.5,1363.55,22960845,3.954912e+10,486972,20.522
1119,2024-07-03,EQ,1791.00,1794.00,1764.65,1730.60,1767.70,1768.65,1776.87,1794.0,1363.55,61608901,1.094709e+11,846404,20.973
1120,2024-07-04,EQ,1759.75,1759.75,1724.85,1768.65,1728.00,1727.15,1735.22,1794.0,1363.55,22924329,3.977872e+10,405416,20.481


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1122 non-null   datetime64[ns]
 1   Series      1122 non-null   object        
 2   Open        1122 non-null   float64       
 3   High        1122 non-null   float64       
 4   Low         1122 non-null   float64       
 5   Prev Close  1122 non-null   float64       
 6   LTP         1122 non-null   float64       
 7   Close       1122 non-null   float64       
 8   VWAP        1122 non-null   float64       
 9   52W H       1122 non-null   float64       
 10  52W L       1122 non-null   float64       
 11  Volume      1122 non-null   int64         
 12  Value       1122 non-null   float64       
 13  #Trades     1122 non-null   int64         
 14  PE          1122 non-null   float64       
dtypes: datetime64[ns](1), float64(11), int64(2), object(1)
memory usage: 187

## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Prev Close']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df = stock_df.drop(columns = ['Series', 'Prev Close'])
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh
310,2021-03-26,1494.00,1499.00,1474.00,1497.55,1491.30,1486.80,1641.00,738.75,6010629,8.936580e+09,186603,30.006,25.00,1,0,0
437,2021-10-01,1583.00,1589.00,1565.25,1585.65,1582.70,1577.48,1641.00,1042.65,5564298,8.777583e+09,145318,27.411,23.75,0,0,0
812,2023-04-06,1648.55,1669.20,1647.10,1666.00,1666.35,1659.15,1722.10,1271.60,22318672,3.702998e+10,233336,20.213,22.10,1,0,0
541,2022-03-03,1381.00,1392.00,1362.90,1373.35,1371.00,1377.34,1725.00,1353.00,8763961,1.207094e+10,355777,23.744,29.10,0,0,0
136,2020-07-20,1135.00,1152.90,1124.15,1130.00,1133.05,1138.35,2349.85,738.75,32562064,3.706709e+10,505889,22.798,28.75,1,0,0
680,2022-09-23,1472.35,1475.00,1437.00,1447.00,1446.15,1451.15,1725.00,1271.60,7903126,1.146862e+10,254765,21.075,38.00,0,0,0
44,2020-03-03,1175.00,1185.35,1168.00,1182.50,1181.80,1178.14,2503.30,1084.00,11184775,1.317718e+10,233028,28.824,17.35,1,0,0
207,2020-10-28,1228.00,1232.00,1201.50,1210.30,1209.60,1214.22,1305.50,738.75,8285625,1.006059e+10,202965,24.338,30.50,0,0,0
618,2022-06-24,1338.55,1361.65,1338.55,1355.00,1353.80,1354.94,1725.00,1271.60,4006716,5.428848e+09,265751,19.729,23.10,1,0,0
955,2023-11-03,1488.00,1491.40,1481.65,1484.50,1483.75,1487.25,1757.50,1460.25,9047670,1.345615e+10,226806,17.998,9.75,1,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      52.05
Is52WLow      1.60
Is52WHigh     2.41
dtype: float64

In [9]:
stock_df.shape

(1122, 17)

### Streak

In [10]:
stock_df["Streak"] = (stock_df["IsGreen"] != stock_df["IsGreen"].shift(1)).cumsum()
stock_df["Streak"] = stock_df.groupby("Streak").cumcount() + 1
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,2024-07-01,1680.00,1707.30,1680.00,1704.20,1705.20,1699.74,1757.5,1363.55,10593512,1.800619e+10,355235,20.221,27.30,1,0,0,1
1118,2024-07-02,1715.00,1734.90,1702.75,1728.00,1730.60,1722.46,1757.5,1363.55,22960845,3.954912e+10,486972,20.522,32.15,1,0,0,2
1119,2024-07-03,1791.00,1794.00,1764.65,1767.70,1768.65,1776.87,1794.0,1363.55,61608901,1.094709e+11,846404,20.973,29.35,1,0,1,3
1120,2024-07-04,1759.75,1759.75,1724.85,1728.00,1727.15,1735.22,1794.0,1363.55,22924329,3.977872e+10,405416,20.481,34.90,0,0,0,1


### Date based columns

In [11]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
839,2023-05-18,1648.00,1653.95,1640.00,1644.50,1645.00,1646.05,1734.45,1271.60,24823471,4.086056e+10,278185,19.954,13.95,1,0,0,1,18,5,2023,3,138,2,1
538,2022-02-25,1439.95,1465.90,1432.85,1456.30,1456.10,1454.62,1725.00,1353.00,9159327,1.332331e+10,204609,25.218,33.05,1,0,0,1,25,2,2022,4,56,1,1
457,2021-11-01,1585.00,1611.00,1583.55,1609.00,1605.30,1594.91,1725.00,1177.50,4061160,6.477168e+09,154509,27.802,27.45,1,0,0,1,1,11,2021,0,305,4,3
1012,2024-01-25,1452.50,1457.35,1419.40,1440.70,1434.90,1434.16,1757.50,1380.25,49470926,7.094916e+10,787699,17.405,37.95,0,0,0,1,25,1,2024,3,25,1,1
410,2021-08-24,1530.00,1564.50,1527.45,1558.00,1558.85,1550.18,1641.00,1025.00,9190657,1.424720e+10,214219,26.998,37.05,1,0,0,3,24,8,2021,1,236,3,1
482,2021-12-08,1536.00,1555.05,1534.00,1553.50,1553.80,1545.20,1725.00,1342.00,7647767,1.181731e+10,229215,26.910,21.05,1,0,0,2,8,12,2021,2,342,4,1
261,2021-01-14,1471.15,1488.00,1456.00,1474.00,1468.75,1468.80,1496.90,738.75,6148583,9.031062e+09,187531,29.552,32.00,0,0,0,2,14,1,2021,3,14,1,1
139,2020-07-23,1125.00,1143.90,1116.25,1129.80,1130.40,1133.04,2349.85,738.75,10074805,1.141519e+10,179616,22.744,27.65,1,0,0,1,23,7,2020,3,205,3,1
1033,2024-02-26,1413.55,1434.85,1413.55,1421.95,1422.30,1426.02,1757.50,1363.55,11753435,1.676058e+10,381979,17.253,21.30,1,0,0,2,26,2,2024,0,57,1,3
601,2022-06-01,1380.00,1400.45,1379.15,1396.10,1394.85,1391.06,1725.00,1278.30,6045948,8.410276e+09,152770,20.327,21.30,1,0,0,1,1,6,2022,2,152,2,1


In [12]:
stock_df.shape

(1122, 25)

### Moving averages

#### `Close`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,1278.60,1278.60,1278.60,1278.60,1278.60
1,1286.75,1282.68,1282.68,1282.68,1282.68
2,1268.40,1277.92,1277.92,1277.92,1277.92
3,1240.95,1268.68,1268.68,1268.68,1268.68
4,1260.60,1267.06,1267.06,1267.06,1267.06
...,...,...,...,...,...
1117,1705.20,1690.88,1643.27,1579.87,1541.09
1118,1730.60,1700.14,1654.56,1588.69,1545.23
1119,1768.65,1713.89,1668.15,1599.01,1549.24
1120,1727.15,1716.15,1678.35,1607.94,1552.20


#### `Range`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,9.40,9.40,9.40,9.40,9.40
1,9.00,9.20,9.20,9.20,9.20
2,21.40,13.27,13.27,13.27,13.27
3,25.80,16.40,16.40,16.40,16.40
4,19.20,16.96,16.96,16.96,16.96
...,...,...,...,...,...
1117,27.30,29.43,27.94,30.76,27.56
1118,32.15,29.78,28.55,31.70,27.69
1119,29.35,29.78,29.25,32.02,27.75
1120,34.90,28.21,30.12,32.61,27.92


#### `VWAP`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,1276.64,1276.64,1276.64,1276.64,1276.64
1,1284.56,1280.60,1280.60,1280.60,1280.60
2,1270.48,1277.23,1277.23,1277.23,1277.23
3,1247.24,1269.73,1269.73,1269.73,1269.73
4,1261.48,1268.08,1268.08,1268.08,1268.08
...,...,...,...,...,...
1117,1699.74,1688.84,1641.36,1577.69,1540.26
1118,1722.46,1697.96,1651.73,1586.25,1544.22
1119,1776.87,1713.95,1665.90,1596.81,1548.49
1120,1735.22,1718.26,1676.45,1606.11,1551.65


#### `Volume`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1836849,1836849,1836849,1836849,1836849
1,3068583,2452716,2452716,2452716,2452716
2,5427775,3444402,3444402,3444402,3444402
3,5445093,3944575,3944575,3944575,3944575
4,7362247,4628109,4628109,4628109,4628109
...,...,...,...,...,...
1117,10593512,21787978,20330930,21564760,19477741
1118,22960845,21380187,21113185,22312503,19480552
1119,61608901,28142530,24235601,23636633,19766241
1120,22924329,26094466,24454541,23734173,19656181


#### `Value`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,2.345001e+09,2345000988,2345000988,2345000988,2345000988
1,3.941792e+09,3143396262,3143396262,3143396262,3143396262
2,6.895886e+09,4394226092,4394226092,4394226092,4394226092
3,6.791348e+09,4993506527,4993506527,4993506527,4993506527
4,9.287302e+09,5852265530,5852265530,5852265530,5852265530
...,...,...,...,...,...
1117,1.800619e+10,36830141107,33568698980,34103306599,30132175903
1118,3.954912e+10,36362952601,35032512679,35395789340,30227198113
1119,1.094709e+11,48606985959,40790042612,37979692251,30924717627
1120,3.977872e+10,45213784428,41377137957,38334910813,30827184461


#### `#Trades`

In [18]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,46625,46625,46625,46625,46625
1,104570,75597,75597,75597,75597
2,157066,102753,102753,102753,102753
3,155007,115817,115817,115817,115817
4,189026,130458,130458,130458,130458
...,...,...,...,...,...
1117,355235,452245,427346,426357,399785
1118,486972,463445,438873,442056,401547
1119,846404,528991,464713,456492,405017
1120,405416,488238,469002,456860,404336


### Target columns

In [19]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,1240.95,1282.70,1240.85,1240.60
1,1260.60,1286.00,1244.85,1249.00
2,1257.30,1289.50,1244.55,1241.40
3,1271.40,1284.25,1213.20,1219.35
4,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...
1117,1727.15,NaN,NaN,NaN
1118,1648.10,NaN,NaN,NaN
1119,NaN,NaN,NaN,NaN
1120,NaN,NaN,NaN,NaN


### Saving processed data

In [20]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1,1,1,2020,2,1,1,1,1278.60,1278.60,1278.60,1278.60,9.40,9.40,9.40,9.40,1276.64,1276.64,1276.64,1276.64,1836849,1836849,1836849,1836849,2345000988,2345000988,2345000988,2345000988,46625,46625,46625,46625,1240.95,1282.70,1240.85,1240.60
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2,2,1,2020,3,2,1,1,1282.68,1282.68,1282.68,1282.68,9.20,9.20,9.20,9.20,1280.60,1280.60,1280.60,1280.60,2452716,2452716,2452716,2452716,3143396262,3143396262,3143396262,3143396262,75597,75597,75597,75597,1260.60,1286.00,1244.85,1249.00
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1,3,1,2020,4,3,1,1,1277.92,1277.92,1277.92,1277.92,13.27,13.27,13.27,13.27,1277.23,1277.23,1277.23,1277.23,3444402,3444402,3444402,3444402,4394226092,4394226092,4394226092,4394226092,102753,102753,102753,102753,1257.30,1289.50,1244.55,1241.40
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2,6,1,2020,0,6,1,3,1268.68,1268.68,1268.68,1268.68,16.40,16.40,16.40,16.40,1269.73,1269.73,1269.73,1269.73,3944575,3944575,3944575,3944575,4993506527,4993506527,4993506527,4993506527,115817,115817,115817,115817,1271.40,1284.25,1213.20,1219.35
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1,7,1,2020,1,7,1,1,1267.06,1267.06,1267.06,1267.06,16.96,16.96,16.96,16.96,1268.08,1268.08,1268.08,1268.08,4628109,4628109,4628109,4628109,5852265530,5852265530,5852265530,5852265530,130458,130458,130458,130458,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,2024-07-01,1680.00,1707.30,1680.00,1704.20,1705.20,1699.74,1757.5,1363.55,10593512,1.800619e+10,355235,20.221,27.30,1,0,0,1,1,7,2024,0,183,3,3,1690.88,1643.27,1579.87,1541.09,29.43,27.94,30.76,27.56,1688.84,1641.36,1577.69,1540.26,21787978,20330930,21564760,19477741,36830141107,33568698980,34103306599,30132175903,452245,427346,426357,399785,1727.15,NaN,NaN,NaN
1118,2024-07-02,1715.00,1734.90,1702.75,1728.00,1730.60,1722.46,1757.5,1363.55,22960845,3.954912e+10,486972,20.522,32.15,1,0,0,2,2,7,2024,1,184,3,1,1700.14,1654.56,1588.69,1545.23,29.78,28.55,31.70,27.69,1697.96,1651.73,1586.25,1544.22,21380187,21113185,22312503,19480552,36362952601,35032512679,35395789340,30227198113,463445,438873,442056,401547,1648.10,NaN,NaN,NaN
1119,2024-07-03,1791.00,1794.00,1764.65,1767.70,1768.65,1776.87,1794.0,1363.55,61608901,1.094709e+11,846404,20.973,29.35,1,0,1,3,3,7,2024,2,185,3,1,1713.89,1668.15,1599.01,1549.24,29.78,29.25,32.02,27.75,1713.95,1665.90,1596.81,1548.49,28142530,24235601,23636633,19766241,48606985959,40790042612,37979692251,30924717627,528991,464713,456492,405017,NaN,NaN,NaN,NaN
1120,2024-07-04,1759.75,1759.75,1724.85,1728.00,1727.15,1735.22,1794.0,1363.55,22924329,3.977872e+10,405416,20.481,34.90,0,0,0,1,4,7,2024,3,186,3,1,1716.15,1678.35,1607.94,1552.20,28.21,30.12,32.61,27.92,1718.26,1676.45,1606.11,1551.65,26094466,24454541,23734173,19656181,45213784428,41377137957,38334910813,30827184461,488238,469002,456860,404336,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [21]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [22]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,1.958,0.848,1.0,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,0.998,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,1.945,0.842,1.0,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.007,0.998,0.995,0.995,0.995,0.995,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.974,0.855,1.0,1.008,1.008,1.008,1.008,0.017,0.010,0.010,0.010,0.010,1.002,1.007,1.007,1.007,1.007,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,2.017,0.874,1.0,1.022,1.022,1.022,1.022,0.021,0.013,0.013,0.013,0.013,1.005,1.023,1.023,1.023,1.023,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.986,0.860,1.0,1.005,1.005,1.005,1.005,0.015,0.013,0.013,0.013,0.013,1.001,1.006,1.006,1.006,1.006,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,0.985,1.001,0.985,0.999,1.031,0.800,1.0,0.992,0.964,0.927,0.904,0.016,0.017,0.016,0.018,0.016,0.997,0.990,0.963,0.925,0.903,1.013,NaN,NaN,NaN
1118,0.991,1.002,0.984,0.998,1.016,0.788,1.0,0.982,0.956,0.918,0.893,0.019,0.017,0.016,0.018,0.016,0.995,0.981,0.954,0.917,0.892,0.952,NaN,NaN,NaN
1119,1.013,1.014,0.998,0.999,1.014,0.771,1.0,0.969,0.943,0.904,0.876,0.017,0.017,0.017,0.018,0.016,1.005,0.969,0.942,0.903,0.876,NaN,NaN,NaN,NaN
1120,1.019,1.019,0.999,1.000,1.039,0.789,1.0,0.994,0.972,0.931,0.899,0.020,0.016,0.017,0.019,0.016,1.005,0.995,0.971,0.930,0.898,NaN,NaN,NaN,NaN


### Volume based columns

In [23]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [24]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.799,0.799,0.799,0.799
2,1.0,0.635,0.635,0.635,0.635
3,1.0,0.724,0.724,0.724,0.724
4,1.0,0.629,0.629,0.629,0.629
...,...,...,...,...,...
1117,1.0,2.057,1.919,2.036,1.839
1118,1.0,0.931,0.920,0.972,0.848
1119,1.0,0.457,0.393,0.384,0.321
1120,1.0,1.138,1.067,1.035,0.857


### Value based columns

In [25]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [26]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.797,0.797,0.797,0.797
2,1.0,0.637,0.637,0.637,0.637
3,1.0,0.735,0.735,0.735,0.735
4,1.0,0.630,0.630,0.630,0.630
...,...,...,...,...,...
1117,1.0,2.045,1.864,1.894,1.673
1118,1.0,0.919,0.886,0.895,0.764
1119,1.0,0.444,0.373,0.347,0.282
1120,1.0,1.137,1.040,0.964,0.775


### Trade count based columns

In [27]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [28]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.723,0.723,0.723,0.723
2,1.0,0.654,0.654,0.654,0.654
3,1.0,0.747,0.747,0.747,0.747
4,1.0,0.690,0.690,0.690,0.690
...,...,...,...,...,...
1117,1.0,1.273,1.203,1.200,1.125
1118,1.0,0.952,0.901,0.908,0.825
1119,1.0,0.625,0.549,0.539,0.479
1120,1.0,1.204,1.157,1.127,0.997


### Dropping unnecessary columns

In [29]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,0.985,1.001,0.985,0.999,0.997,1.031,0.800,20.221,1,0,0,1,1,7,2024,0,183,3,3,0.992,0.964,0.927,0.904,0.017,0.016,0.018,0.016,0.990,0.963,0.925,0.903,2.057,1.919,2.036,1.839,2.045,1.864,1.894,1.673,1.273,1.203,1.200,1.125,1.013,NaN,NaN,NaN
1118,0.991,1.002,0.984,0.998,0.995,1.016,0.788,20.522,1,0,0,2,2,7,2024,1,184,3,1,0.982,0.956,0.918,0.893,0.017,0.016,0.018,0.016,0.981,0.954,0.917,0.892,0.931,0.920,0.972,0.848,0.919,0.886,0.895,0.764,0.952,0.901,0.908,0.825,0.952,NaN,NaN,NaN
1119,1.013,1.014,0.998,0.999,1.005,1.014,0.771,20.973,1,0,1,3,3,7,2024,2,185,3,1,0.969,0.943,0.904,0.876,0.017,0.017,0.018,0.016,0.969,0.942,0.903,0.876,0.457,0.393,0.384,0.321,0.444,0.373,0.347,0.282,0.625,0.549,0.539,0.479,NaN,NaN,NaN,NaN
1120,1.019,1.019,0.999,1.000,1.005,1.039,0.789,20.481,0,0,0,1,4,7,2024,3,186,3,1,0.994,0.972,0.931,0.899,0.016,0.017,0.019,0.016,0.995,0.971,0.930,0.898,1.138,1.067,1.035,0.857,1.137,1.040,0.964,0.775,1.204,1.157,1.127,0.997,NaN,NaN,NaN,NaN


### Saving standardized data

In [30]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,0.985,1.001,0.985,0.999,0.997,1.031,0.800,20.221,1,0,0,1,1,7,2024,0,183,3,3,0.992,0.964,0.927,0.904,0.017,0.016,0.018,0.016,0.990,0.963,0.925,0.903,2.057,1.919,2.036,1.839,2.045,1.864,1.894,1.673,1.273,1.203,1.200,1.125,1.013,NaN,NaN,NaN
1118,0.991,1.002,0.984,0.998,0.995,1.016,0.788,20.522,1,0,0,2,2,7,2024,1,184,3,1,0.982,0.956,0.918,0.893,0.017,0.016,0.018,0.016,0.981,0.954,0.917,0.892,0.931,0.920,0.972,0.848,0.919,0.886,0.895,0.764,0.952,0.901,0.908,0.825,0.952,NaN,NaN,NaN
1119,1.013,1.014,0.998,0.999,1.005,1.014,0.771,20.973,1,0,1,3,3,7,2024,2,185,3,1,0.969,0.943,0.904,0.876,0.017,0.017,0.018,0.016,0.969,0.942,0.903,0.876,0.457,0.393,0.384,0.321,0.444,0.373,0.347,0.282,0.625,0.549,0.539,0.479,NaN,NaN,NaN,NaN
1120,1.019,1.019,0.999,1.000,1.005,1.039,0.789,20.481,0,0,0,1,4,7,2024,3,186,3,1,0.994,0.972,0.931,0.899,0.016,0.017,0.019,0.016,0.995,0.971,0.930,0.898,1.138,1.067,1.035,0.857,1.137,1.040,0.964,0.775,1.204,1.157,1.127,0.997,NaN,NaN,NaN,NaN
